## Data Preprocessing

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [11]:
# Load the dataset from the CSV file
csv_file = 'data/color_dataset.csv'
df = pd.read_csv(csv_file)

In [13]:
# Display the first few rows of the dataset
print(df.head())

     R    G    B Text_Color
0  165  254  251      black
1  105  243  172      black
2   98   30  170      white
3  116  105  175      white
4   24  204  100      white


In [14]:
# Separate features (RGB values) and labels (Text_Color)
X = df[['R', 'G', 'B']]
y = df['Text_Color']

In [15]:
# Normalize the RGB values (scaling between 0 and 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_normalized = pd.DataFrame(X_scaled, columns=X.columns)

In [16]:
# Convert text labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [17]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)

In [18]:
# Display the shape of training and testing sets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (776537, 3), y_train shape: (776537,)
X_test shape: (194135, 3), y_test shape: (194135,)


## Model Building

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
# Define the neural network architecture
class ColorPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ColorPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [21]:
# Set hyperparameters
input_size = 3  # Input size (R, G, B values)
hidden_size = 16  # Number of neurons in the hidden layer
output_size = 1  # Output size (binary classification)

In [22]:
# Create the model instance
model = ColorPredictor(input_size, hidden_size, output_size)

In [23]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training

In [24]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape for BCELoss

In [25]:
# Define number of epochs and batch size
num_epochs = 100
batch_size = 64

In [26]:
# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i in range(0, len(X_train_tensor), batch_size):
        # Extract batch
        inputs = X_train_tensor[i:i+batch_size]
        labels = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Print average loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / (len(X_train_tensor) / batch_size):.4f}')

Epoch [1/100], Loss: 0.0946
Epoch [2/100], Loss: 0.0250
Epoch [3/100], Loss: 0.0182
Epoch [4/100], Loss: 0.0150
Epoch [5/100], Loss: 0.0130
Epoch [6/100], Loss: 0.0116
Epoch [7/100], Loss: 0.0106
Epoch [8/100], Loss: 0.0098
Epoch [9/100], Loss: 0.0092
Epoch [10/100], Loss: 0.0086
Epoch [11/100], Loss: 0.0082
Epoch [12/100], Loss: 0.0078
Epoch [13/100], Loss: 0.0075
Epoch [14/100], Loss: 0.0072
Epoch [15/100], Loss: 0.0069
Epoch [16/100], Loss: 0.0067
Epoch [17/100], Loss: 0.0065
Epoch [18/100], Loss: 0.0063
Epoch [19/100], Loss: 0.0061
Epoch [20/100], Loss: 0.0059
Epoch [21/100], Loss: 0.0058
Epoch [22/100], Loss: 0.0056
Epoch [23/100], Loss: 0.0055
Epoch [24/100], Loss: 0.0054
Epoch [25/100], Loss: 0.0053
Epoch [26/100], Loss: 0.0052
Epoch [27/100], Loss: 0.0051
Epoch [28/100], Loss: 0.0050
Epoch [29/100], Loss: 0.0049
Epoch [30/100], Loss: 0.0048
Epoch [31/100], Loss: 0.0047
Epoch [32/100], Loss: 0.0046
Epoch [33/100], Loss: 0.0046
Epoch [34/100], Loss: 0.0045
Epoch [35/100], Loss: 0

## Evaluation

In [27]:
# Convert test data to PyTorch tensors
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape for BCELoss

In [28]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

    # Convert outputs to binary predictions (0 or 1)
    predicted_labels = (test_outputs > 0.5).float()

    # Calculate accuracy
    correct = (predicted_labels == y_test_tensor).sum().item()
    total = y_test_tensor.size(0)
    accuracy = correct / total

    print(f'Test Loss: {test_loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%')

Test Loss: 0.0040, Accuracy: 99.76%


## Inference

In [29]:
# Function to predict text color given a background color
def predict_text_color(model, scaler, color):
    # Normalize the input color using the same scaler used for training data
    normalized_color = scaler.transform([color])

    # Convert the normalized color to a PyTorch tensor
    input_tensor = torch.tensor(normalized_color, dtype=torch.float32)

    # Perform prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        predicted_label = 'white' if output > 0.5 else 'black'

    return predicted_label

In [32]:
# Example background color (R, G, B values)
background_color = [180, 105, 178]  # Change this color as needed

# Use the trained model and scaler for inference
predicted_text_color = predict_text_color(model, scaler, background_color)
print(f'Predicted text color for background color {background_color}: {predicted_text_color}')

Predicted text color for background color [180, 105, 178]: black


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


## Export the Model

In [33]:
# Save the model state dictionary
torch.save(model.state_dict(), 'color_predictor_model.pth')

## Import Model for Future API usage

In [44]:
# Load the saved model state dictionary
model.load_state_dict(torch.load('color_predictor_model.pth'))
model.eval()  # Set the model to evaluation mode


ColorPredictor(
  (fc1): Linear(in_features=3, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [45]:
# Example background color (R, G, B values)
background_color = [0, 0, 0]  # Change this color as needed

# Use the trained model and scaler for inference
predicted_text_color = predict_text_color(model, scaler, background_color)
print(f'Predicted text color for background color {background_color}: {predicted_text_color}')

Predicted text color for background color [0, 0, 0]: white


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
